In [11]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os

from pprint import pprint
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

In [15]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")
print(mnemonic)

neglect rice alien toe actor sorry chuckle hazard tortoise hawk dumb weapon


In [16]:
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))
# enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [14]:
# Import constants.py and necessary functions from bit and web3
from constants import *

In [17]:
# Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)


In [18]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

pprint(coins)
print(coins["eth"][0]['privkey'])

{'btc-test': [{'address': 'mre2c3B9VGrg2Ts2ic7bLatciD3ZE21Jrj',
               'index': 0,
               'path': "m/44'/1'/0'/0/0",
               'privkey': 'cRWxwAFkQkPnhvxq5JZQ2jGwF8UUaPtDEsx2LvWcevJrpewHuKmT',
               'pubkey': '03d7baa0b273a4838957a543ee66eb96dcab7fb339484f7f76a6f6cbc27feeed47',
               'pubkeyhash': '79ffe4aa3e39d055fef4d270e44f702d8438c252',
               'xprv': 'tprv8k8KPAmvVXHVxJssqjriZbCiMTnukMQ5mVUtUobw2WpJ7zPmpP6TG3cQW6cSALA4VWiN9c3TrwWZW8w97auhh6wwohcvKAyCv2uJqBZ6Gea',
               'xpub': 'tpubDGpMXapAdtyAqmufjPXJxzrpvVJqugazLo5fmKeESncgxUeYSmv3SYEGgGFNHK1vizh1JP42qxim3gGQL7gxpMsbqHrXDu4bEqyWUFJxXVL'},
              {'address': 'mn67RWrhCjZcxJ2fzFKLe4gMSptWvrXwcU',
               'index': 1,
               'path': "m/44'/1'/0'/0/1",
               'privkey': 'cTU3uQxAarPFo8hGKmeu3FVNWnBF8rgy6ZATFWMqMnwJFwzyqZ6V',
               'pubkey': '035da9b550d6b0615c028e819fb02ee4a85ecb644b29375213fb112dd4bc708179',
               'pubkeyhash': '

In [19]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
   
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [20]:
eth_acc = priv_key_to_account(ETH,"0xd6b45057a4f11f7591944e4721bb7e5128ee9e79d6e1f72ea89e03e5c2e676ec")
print(eth_acc)

In [21]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    
    global trx_data
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 wei
        gasEstimate = w3.eth.estimateGas(
            {"from": account, "to": to, "amount": value }
        )
        trx_data = {
            "to": to,
            "from": account,
            "amount": value,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [22]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

In [23]:
account = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'], )

In [24]:
send_tx(BTCTEST, account, 'mn67RWrhCjZcxJ2fzFKLe4gMSptWvrXwcU', '0.0001')